In [238]:
# Imports of necessary libraries
import os
import pandas as pd
import numpy as np
import numpy as np
import pandas as  pd


# for LDA

import re
# Gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary, TextCorpus

# spaCy for preprocessing
import spacy

# basic plotting tools
import matplotlib.pyplot as plt
import seaborn as sns

# Lda plotting
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

In [136]:
#import function from python file
import preprocessing

# Try with gensim

### Get datasets

In [4]:
df_movies, df_summaries = preprocessing.get_datasets()
df_genres = preprocessing.get_movie_genres_dataframe(df_movies=df_movies)

if os.path.exists('df_fictional_summaries.csv'):
    df_fictional_summaries = pd.load('df_fictional_summaries.csv')
else:
    df_fictional_summaries = preprocessing.get_fictional_summaries_subset(df_genres=df_genres, df_movies=df_movies, df_summaries=df_summaries)
    df_fictional_summaries.to_csv('df_fictional_summaries.csv')

if os.path.exists("preprocessed_fictional_summaries.csv"):
    df_preprocessed_fictional_summaries = pd.read_csv("preprocessed_fictional_summaries.csv")

   Wikipedia_movie_ID Freebase_genre_ID      movie_genre
0              975900         /m/01jfsb         Thriller
1              975900          /m/06n90  Science Fiction
2              975900          /m/03npn           Horror
3              975900         /m/03k9fj        Adventure
4              975900          /m/0fdjb     Supernatural
The total number of movies referred to as fictional is 5366, corresponding to 6.56% of whole movies.


### Tokenize

#### a) Method 1

In [215]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Tokenize documents using spaCy
tokenized_docs = []

for doc in nlp.pipe(df_fictional_summaries['movie_summary'], disable=["parser", "textcat"], batch_size=100, n_process=4):
    tokens = [token.text.lower() for token in doc if token.is_alpha and not token.is_stop and token.pos_ not in ['PROPN']] ## Process P1
    tokenized_docs.append(tokens)

# takes around 2min without lemmatization
# takes around 2min with lemma 

In [216]:
# Create a Gensim dictionary
dictionary = Dictionary(tokenized_docs)

# Filter out extreme cases (optional)
max_freq = 0.50 # 50% reduces only 2-3 words, 30% ~12 tokens
min_wordcount = 5
dictionary.filter_extremes(no_below=min_wordcount, no_above=max_freq)

# Create a Gensim corpus
corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]

In [218]:
from gensim.models import TfidfModel

#Create a TF-IDF model for better topic modeling results
tfidf_model = TfidfModel(corpus, normalize=False)
tfidf_corpus = tfidf_model[corpus]

#### b) Method sklearn

In [121]:
from gensim.models import LdaMulticore
from gensim import matutils
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.preprocessing import normalize


In [ ]:
# pipeline = ["tok2vec", "tagger", "attribute_ruler", "parser", "ner", 'lemmatizer']
# nlp_exclusif = spacy.load("en_core_web_sm", enable=pipeline)

# # Assuming you have a list of preprocessed documents
# preprocessed_documents = pd.read_csv("preprocessed_fictional_summaries.csv")['preprocessed_summary']

# def custom_tokenizer(text):
#     doc = nlp_exclusif(text)
#     tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop and token.pos_ not in ['PROPN']]
#     return tokens

# # Convert documents to TF-IDF matrix
# vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_df=0.75, min_df=5)
# token_matrix = vectorizer.fit_transform(preprocessed_documents)

# # This does Count + Tfidf + normalization
# tfidf_vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, norm='l2', max_df=0.70, min_df=10)
# tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_documents)


In [177]:
# Create a Gensim dictionary
dictionary = Dictionary(tokenized_docs)
dictionary.filter_extremes(no_above=0.5, no_below=5)

vectorizer = CountVectorizer(vocabulary=dictionary.token2id.keys())
bow_matrix = vectorizer.fit_transform([' '.join(doc) for doc in tokenized_docs])
bow_matrix.shape

c:\Users\gezas\.conda\envs\ada\lib\site-packages\sklearn\feature_extraction\text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


(3941, 12046)

In [178]:
tfidf_transformer = TfidfTransformer(norm = 'l2', use_idf=True)
tfidf_matrix = tfidf_transformer.fit_transform(bow_matrix)

In [179]:
# Convert the scipy sparse matrix to a gensim corpus
corpus = matutils.Sparse2Corpus(tfidf_matrix, documents_columns=False)

#### c) Gensim LDA

In [226]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of chunks: %d' % len(tfidf_corpus))

Number of unique tokens: 11783
Number of chunks: 3941


In [228]:
# LDA model
params = {'passes': 10, 'random_state': 7}
lda_model = LdaMulticore(corpus=tfidf_corpus, num_topics=10, id2word=dictionary, workers=6,
                passes=params['passes'], random_state=params['random_state'])

# Takes around 1min30

In [229]:
lda_model.show_topics(num_words=5)

[(0,
  '0.002*"creature" + 0.002*"father" + 0.002*"apartment" + 0.002*"man" + 0.002*"police"'),
 (1,
  '0.003*"team" + 0.002*"military" + 0.002*"planet" + 0.002*"alien" + 0.002*"base"'),
 (2,
  '0.007*"ship" + 0.007*"crew" + 0.004*"alien" + 0.003*"aliens" + 0.003*"space"'),
 (3,
  '0.004*"vampire" + 0.003*"vampires" + 0.002*"spell" + 0.002*"father" + 0.002*"magic"'),
 (4,
  '0.003*"robot" + 0.002*"game" + 0.002*"robots" + 0.002*"portal" + 0.002*"apes"'),
 (5,
  '0.004*"monster" + 0.003*"island" + 0.002*"village" + 0.002*"house" + 0.002*"children"'),
 (6,
  '0.002*"machine" + 0.002*"clone" + 0.002*"children" + 0.002*"program" + 0.002*"world"'),
 (7,
  '0.002*"school" + 0.002*"soul" + 0.002*"man" + 0.002*"time" + 0.002*"life"'),
 (8,
  '0.005*"dragon" + 0.004*"castle" + 0.003*"sword" + 0.002*"fairies" + 0.002*"dragons"'),
 (9,
  '0.003*"parents" + 0.002*"children" + 0.002*"school" + 0.002*"house" + 0.002*"tells"')]

In [230]:
lda_model.show_topic(1,20)

[('team', 0.0033741728),
 ('military', 0.0023656476),
 ('planet', 0.0023161843),
 ('alien', 0.0022457736),
 ('base', 0.0022366461),
 ('nuclear', 0.0022297218),
 ('scientists', 0.0020647114),
 ('ship', 0.001933678),
 ('bomb', 0.0018494243),
 ('government', 0.0017682345),
 ('space', 0.0017661825),
 ('attack', 0.0016600527),
 ('group', 0.0016309655),
 ('creature', 0.0016039577),
 ('city', 0.0014985545),
 ('facility', 0.0014487285),
 ('helicopter', 0.0014313638),
 ('human', 0.0013964404),
 ('time', 0.0013838297),
 ('machine', 0.00137827)]

In [231]:
# Get the top words for each topic
top_words_per_topic = []
for topic_id in range(lda_model.num_topics):
    top_words = lda_model.show_topic(topic_id, topn=20)  # Adjust topn as needed
    top_words_per_topic.append(top_words)

# Flatten the list of top words
all_top_words = [word for topic_words in top_words_per_topic for word, _ in topic_words]

# Count the occurrences of each word
from collections import Counter
word_counts = Counter(all_top_words)

# Find the most common words
most_common_words = word_counts.most_common(10)  # Adjust the number as needed

# Display the most common words
for word, count in most_common_words:
    print(f"{word}: {count}")

father: 6
tells: 6
time: 5
creature: 4
house: 4
film: 4
man: 3
mother: 3
home: 3
planet: 3


# Plot results

In [232]:
# Can run this to load model
if lda_model is None:
    lda_model = LdaMulticore.load("LDA_models/lda_model_min_5_max_05_20230912_plustfidf.model")
    dictionary = Dictionary.load("LDA_models/dictionary_processed_P1.txt")

In [246]:
import pyLDAvis.gensim_models
# plot topics
data =  pyLDAvis.gensim_models.prepare(lda_model, tfidf_corpus, dictionary)
pyLDAvis.display(data)

In [234]:
lda_model.save("lda_model_min_5_max_05_20230912_plustfidf.model")